In [1]:
import pandas as pd

In [2]:
train = pd.read_csv(r'C:\Users\KAMALI\Desktop\dataset\train.csv',encoding='latin-1')

In [3]:
train.head(10)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
5,6,0,or i just worry too much?
6,7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,8,0,Sunny Again Work Tomorrow :-| ...
8,9,1,handed in my uniform today . i miss you ...
9,10,1,hmmmm.... i wonder how she my number @-)


In [4]:
import numpy as np
import re, string

In [5]:
def remove_pattern(input_txt,pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

In [6]:
train['SentimentText'] = np.vectorize(remove_pattern)(train['SentimentText'],"@[\w]*")

In [7]:
train['SentimentText'] = train['SentimentText'].str.replace("[^a-zA-Z]"," ")

In [8]:
def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

twits = [
    'This is amazing!',
    'ML is the best, yes it is',
    'I am not sure about how this is going to end...'
]

count = CountVectorizer()
bag = count.fit_transform(twits)

count.vocabulary_

{'this': 13,
 'is': 7,
 'amazing': 2,
 'ml': 9,
 'the': 12,
 'best': 3,
 'yes': 15,
 'it': 8,
 'am': 1,
 'not': 10,
 'sure': 11,
 'about': 0,
 'how': 6,
 'going': 5,
 'to': 14,
 'end': 4}

In [10]:
bag.toarray()

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]], dtype=int64)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(train['SentimentText'])

In [12]:
from sklearn.model_selection import train_test_split

# split the dataset in train and test
X = train['SentimentText']
y = train['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [13]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
rf = RandomForestClassifier()
t0 = time.time()review = "i hate you bitch"
review_vector = vectorizer.transform([review]) # vectorizing
print(rf.predict(review_vector))
res = rf.fit(train_vectors, train['Sentiment'])
t1 = time.time()
prediction_linear = rf.predict(train_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(train['Sentiment'], prediction_linear, output_dict=True)
print('positive: ', report['1'])
print('negative: ', report['0'])

Training time: 330.790392s; Prediction time: 10.075134s
positive:  {'precision': 0.9910901375217904, 'recall': 0.9969534335866235, 'f1-score': 0.9940131393048884, 'support': 56457}
negative:  {'precision': 0.996018334182138, 'recall': 0.988376366810622, 'f1-score': 0.9921826357661708, 'support': 43532}


In [14]:
review = "the world is so scared right now"
review_vector = vectorizer.transform([review]) # vectorizing
print(rf.predict(review_vector))

[1]


In [15]:
review = "i hate you bitch"
review_vector = vectorizer.transform([review]) # vectorizing
print(rf.predict(review_vector))

[0]


In [16]:
import pickle

In [18]:
filename = 'rf_model'
outfile = open(filename,'wb')

In [19]:
pickle.dump(res,outfile)
outfile.close()

In [20]:
import pickle

In [21]:
filename = 'rf_model'
infile = open(filename,'rb')
loaded_model = pickle.load(infile)
infile.close()

In [23]:
review = "that was not upto the level"
review_vector = vectorizer.transform([review]) # vectorizing
print(rf.predict(review_vector))

[0]


In [24]:
review = "i was so curious"
review_vector = vectorizer.transform([review]) # vectorizing
print(rf.predict(review_vector))

[0]


In [25]:
review = "just get out from here"
review_vector = vectorizer.transform([review]) # vectorizing
print(rf.predict(review_vector))

[1]
